En este momento tenemos dos fuentes de datos:

1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. 

 - Cargaremos los dos ficheros de datos

 - Del dataframe de los ataques nos quedaremos solo con las filas de los países USA, AUSTRALIA, SOUTH AFRICA, NEW ZEALAND, PAPUA NEW GUINEA

 - Del dataframe de los datos climáticos seleccionaremos todas las columnas.

 - Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

  - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país

 - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

 - # os recomendamos resetear el index del dataframe de los datos climáticos para que no se repitan los nombres de las columnas.


# El primer problema al que nos podemos enfrentar es que si vemos los tipos de las columnas vemos que estas columnas son objetos, es decir, strings, lo que hará que trabajar con ellas sea un poco complicado: 
clima.dtypes

timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object

# en Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su tipo correspondiente. En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos de la siguiente forma: 

import ast

clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 


x = clima['wind_profile'].apply(pd.Series)


# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
x = df['rh_profile'].apply(pd.Series) 

# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)

# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 

# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 

 - Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [13]:
import pandas as pd
import requests
import numpy as np

1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer

In [14]:
df =pd.read_csv("../datos/attack_limpio_2.csv", index_col= 0)

In [24]:
df.head(2)

,Unnamed: 0.1,case_number,year,type,country,area,location,activity,name,age,time,species_,href,injury,date,mes,fatal_,sex2,species_tipo,edad
0,0,1800.00.00,1800.0,Unprovoked,SEYCHELLES,St. Anne,NaN,a corsair's boat was overturned,NaN,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,"FATAL, all onboard were killed by sharks",1800,UNKNOWN,y,F,NaN,NaN
1,1,1797.05.28.R,1797.0,Unprovoked,NaN,NaN,NaN,Dropped overboard,child,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,FATAL,Reported May-28-1797,May,y,NaN,NaN,NaN


In [15]:
df2 =pd.read_csv("../datos/paises_meteo.csv", index_col= 0)

In [25]:
df2.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,pais
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 235, 'speed':...",24,-1,11,1007,rain,2,0,290,2,39.78373,-100.445882,USA
1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 8}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 275, 'speed':...",24,-4,10,1008,rain,3,0,300,2,39.78373,-100.445882,USA


 - Del dataframe de los ataques nos quedaremos solo con las filas de los países USA, AUSTRALIA, SOUTH AFRICA, NEW ZEALAND, PAPUA NEW GUINEA

In [42]:
df_paises = df[(df["country"]== "USA") | (df["country"] == "AUSTRALIA") | (df["country"] == "NEW ZEALAND") | (df["country"] == "SOUTH AFRICA") | (df["country"] == "PAPUA NEW GUINEA")] 

In [43]:
df_paises["country"].unique()

array(['AUSTRALIA', 'USA', 'PAPUA NEW GUINEA', 'NEW ZEALAND',
       'SOUTH AFRICA'], dtype=object)

 - Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [44]:
df2.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed', 'latitud', 'longitud', 'pais'],
      dtype='object')

In [50]:
df2["pais"].unique()

array(['USA', 'AUSTRALIA', 'SOUTH AFRICA', 'NEW ZEALAND',
       'PAPUA NEW GUINEA'], dtype=object)

 - Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

In [51]:
df2.rename(columns={"pais":"country"}, inplace=True)

In [52]:
df2.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed', 'latitud', 'longitud', 'country'],
      dtype='object')

In [63]:
df_final = pd.merge(left = df_paises, right= df2, how= "left", left_on= "country", right_on= "country")
df_final.head(2)

,Unnamed: 0.1,case_number,year,type,country,area,location,activity,name,age,...,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
0,3,1791.00.00,1791.0,Unprovoked,AUSTRALIA,New South Wales,Port Jackson,NaN,"female, an Australian aboriginal",NaN,...,-1,12,1017,rain,2,0,190,3,-24.776109,134.755
1,3,1791.00.00,1791.0,Unprovoked,AUSTRALIA,New South Wales,Port Jackson,NaN,"female, an Australian aboriginal",NaN,...,-1,12,1020,rain,3,0,160,3,-24.776109,134.755


In [69]:
df_final["country"].unique()

array(['AUSTRALIA', 'USA', 'PAPUA NEW GUINEA', 'NEW ZEALAND',
       'SOUTH AFRICA'], dtype=object)